# 🔹 **UNet con LeakyReLU, Dropout y ConvTranspose2d**

Este bloque de código define un **modelo UNet** más profundo, optimizado para segmentación de imágenes.  
A continuación se detallan los componentes:

### 1️⃣ **DoubleConv**
- Bloque de 2 convoluciones 2D con kernel 3x3.
- Activación **LeakyReLU** con `negative_slope=0.1`.
- **Dropout2d** opcional para regularización.
- Entrada: `in_ch` canales.
- Salida: `out_ch` canales.

### 2️⃣ **Encoder (Downsampling)**
- 5 niveles de convoluciones con **MaxPooling**.
- Cada nivel duplica el número de filtros:
  - `64 → 128 → 256 → 512 → 1024`.

### 3️⃣ **Decoder (Upsampling)**
- **ConvTranspose2d** para aumentar la resolución.
- Concatenación con los mapas de características correspondientes del encoder (**skip connections**).
- Cada nivel reduce los filtros a la mitad:
  - `1024 → 512 → 256 → 128 → 64`.

### 4️⃣ **Salida**
- Una convolución final 1x1 que reduce los canales a `out_channels`.
- Activación **sigmoid**, ideal para segmentación binaria

### 5️⃣ **Uso**
```python
model = UNet(in_channels=3, out_channels=1)
output = model(input_tensor)  # input_tensor: [batch, 3, H, W]


In [ ]:
class DoubleConv(nn.Module):
    """Bloque de convoluciones modificado: 2 conv + LeakyReLU + Dropout"""
    def __init__(self, in_ch, out_ch, dropout=0.1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=True),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=True),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Dropout2d(dropout)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super().__init__()
        # Downsampling
        self.down1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.down3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.down4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.down5 = DoubleConv(512, 1024)

        # Upsampling
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv4 = DoubleConv(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv3 = DoubleConv(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv2 = DoubleConv(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv1 = DoubleConv(128, 64)

        # Output
        self.out = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        # Encoder
        c1 = self.down1(x)
        c2 = self.down2(self.pool1(c1))
        c3 = self.down3(self.pool2(c2))
        c4 = self.down4(self.pool3(c3))
        c5 = self.down5(self.pool4(c4))

        # Decoder
        u4 = self.up4(c5)
        u4 = self.conv4(torch.cat([u4, c4], dim=1))
        u3 = self.up3(u4)
        u3 = self.conv3(torch.cat([u3, c3], dim=1))
        u2 = self.up2(u3)
        u2 = self.conv2(torch.cat([u2, c2], dim=1))
        u1 = self.up1(u2)
        u1 = self.conv1(torch.cat([u1, c1], dim=1))

        return torch.sigmoid(self.out(u1))